In [1]:
import sys

sys.path.append(
    "/home/sayem/Desktop/deepHSI"
)  # Replace this with the actual path to your project root

from pathlib import Path

import numpy as np

# Now you should be able to import your custom modules
from src.dataset.components.hyperspectral_dataset import HyperspectralDataset
from src.dataset.components.utils import *

In [2]:
dataset_name = "Salinas"
data_dir = Path("/home/sayem/Desktop/deepHSI/data")
target_folder = data_dir / dataset_name
target_folder.mkdir(parents=True, exist_ok=True)

# Ensure the dataset is downloaded only once for the session
download_dataset(dataset_name, str(target_folder))

# Load the dataset components
img, gt, label_values, ignored_labels, rgb_bands, _ = load_dataset(
    dataset_name, str(target_folder)
)

dataset = HyperspectralDataset(img, gt)

In [3]:
igm, label = dataset[1]

print(igm.shape)
print(label.shape)

torch.Size([1, 204, 5, 5])
torch.Size([])
